In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Extract from zip to csv

import zipfile
import os

!mkdir dataset
base_dir = 'dataset'
path = os.path.join(base_dir, 'archive.zip')

with zipfile.ZipFile(path, 'r') as zip_file:
  zip_file.extractall(base_dir)

mkdir: cannot create directory ‘dataset’: File exists


In [ ]:
# Read CSV file
imdb_df = pd.read_csv(f"{base_dir}/imdb.csv")
imdb_df.head()

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Episodes,Nudity,Violence,Profanity,Alcohol,Frightening
0,No Time to Die,2021,7.6,"107,163","Action, Adventure, Thriller",163,Film,PG-13,-,Mild,Moderate,Mild,Mild,Moderate
1,The Guilty,2021,6.3,"64,375","Crime, Drama, Thriller",90,Film,R,-,None,None,Severe,None,Moderate
2,The Many Saints of Newark,2021,6.4,"27,145","Crime, Drama",120,Film,R,-,Moderate,Severe,Severe,Moderate,Moderate
3,Venom: Let There Be Carnage,2021,6.4,"30,443","Action, Adventure, Sci-Fi",97,Film,PG-13,-,None,Moderate,Moderate,Mild,Moderate
4,Dune,2021,8.3,"84,636","Action, Adventure, Drama",155,Film,PG-13,-,None,Moderate,None,Mild,Moderate


In [ ]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6178 entries, 0 to 6177
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         6178 non-null   object
 1   Date         6178 non-null   int64 
 2   Rate         6178 non-null   object
 3   Votes        6178 non-null   object
 4   Genre        6178 non-null   object
 5   Duration     6178 non-null   object
 6   Type         6178 non-null   object
 7   Certificate  6178 non-null   object
 8   Episodes     6178 non-null   object
 9   Nudity       6178 non-null   object
 10  Violence     6178 non-null   object
 11  Profanity    6178 non-null   object
 12  Alcohol      6178 non-null   object
 13  Frightening  6178 non-null   object
dtypes: int64(1), object(13)
memory usage: 675.8+ KB


In [ ]:
# Select several categorical features

columns = ['Name', 'Genre', 'Certificate', 'Nudity', 'Violence', 'Profanity']

cat_imdb_df = imdb_df[columns]
cat_imdb_df.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,Mild,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,None,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,Moderate,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,None,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,None,Moderate,None


In [ ]:
cat_imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6178 entries, 0 to 6177
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         6178 non-null   object
 1   Genre        6178 non-null   object
 2   Certificate  6178 non-null   object
 3   Nudity       6178 non-null   object
 4   Violence     6178 non-null   object
 5   Profanity    6178 non-null   object
dtypes: object(6)
memory usage: 289.7+ KB


# Brief analysis of the features

In [ ]:
cat_imdb_df["Name"].value_counts()

King Kong                                          5
The Equalizer                                      4
Robin Hood                                         4
Get Shorty                                         4
Lost in Space                                      4
                                                  ..
Master and Commander: The Far Side of the World    1
Gentleman Jack                                     1
Good Grief                                         1
Chicago P.D.                                       1
Star Wars                                          1
Name: Name, Length: 4820, dtype: int64

In [ ]:
cat_imdb_df["Genre"].value_counts()

Comedy                              268
Drama                               259
Crime, Drama, Mystery               220
Comedy, Drama                       199
Drama, Romance                      189
                                   ... 
Documentary, Reality-TV, Romance      1
Comedy, Mystery, Sci-Fi               1
Documentary, History, Reality-TV      1
Musical, Romance                      1
Adventure, Mystery, Thriller          1
Name: Genre, Length: 377, dtype: int64

In [ ]:
cat_imdb_df["Certificate"].value_counts()

R            1885
PG-13        1147
TV-MA         641
TV-14         575
PG            530
None          450
TV-PG         336
Not Rated     231
TV-G           95
G              70
Approved       42
Unrated        40
TV-Y7          34
Passed         24
NC-17          14
TV-Y           13
E              12
(Banned)       11
TV-Y7-FV       11
X               6
GP              6
M               3
M/PG            2
Name: Certificate, dtype: int64

In [ ]:
cat_imdb_df["Nudity"].value_counts()

Mild        2292
None        1459
Moderate    1251
No Rate      707
Severe       469
Name: Nudity, dtype: int64

In [ ]:
cat_imdb_df["Violence"].value_counts()

Moderate    1814
Mild        1703
Severe      1228
No Rate      759
None         674
Name: Violence, dtype: int64

In [ ]:
cat_imdb_df["Profanity"].value_counts()

Mild        2077
Moderate    1646
Severe      1052
No Rate      745
None         658
Name: Profanity, dtype: int64

# Pandas facilities for encoding of categorical features

In [ ]:
cat_imdb_df_1 = cat_imdb_df.copy()

# Categorical features to ints

cat_imdb_df_1[["Nudity", "Violence"]] = cat_imdb_df_1[["Nudity", "Violence"]].astype('category')
cat_imdb_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6178 entries, 0 to 6177
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         6178 non-null   object  
 1   Genre        6178 non-null   object  
 2   Certificate  6178 non-null   object  
 3   Nudity       6178 non-null   category
 4   Violence     6178 non-null   category
 5   Profanity    6178 non-null   object  
dtypes: category(2), object(4)
memory usage: 205.6+ KB


In [ ]:
cat_imdb_df_1["Nudity"] = cat_imdb_df_1["Nudity"].cat.codes
cat_imdb_df_1["Violence"] = cat_imdb_df_1["Violence"].cat.codes
cat_imdb_df_1.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,1,Mild
1,The Guilty,"Crime, Drama, Thriller",R,3,3,Severe
2,The Many Saints of Newark,"Crime, Drama",R,1,4,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,1,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,3,1,None


In [ ]:
cat_imdb_df_1["Nudity"].dtype

dtype('int8')

In [ ]:
# Categorical feature to one-hot

obj_df_new = pd.get_dummies(cat_imdb_df_1, columns=["Profanity"])
obj_df_new.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity_Mild,Profanity_Moderate,Profanity_No Rate,Profanity_None,Profanity_Severe
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,1,1,0,0,0,0
1,The Guilty,"Crime, Drama, Thriller",R,3,3,0,0,0,0,1
2,The Many Saints of Newark,"Crime, Drama",R,1,4,0,0,0,0,1
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,1,0,1,0,0,0
4,Dune,"Action, Adventure, Drama",PG-13,3,1,0,0,0,1,0


In [ ]:
cat_imdb_df_1 = pd.get_dummies(cat_imdb_df_1, columns=["Profanity"], prefix=["prof"])
cat_imdb_df_1.head()

,Name,Genre,Certificate,Nudity,Violence,prof_Mild,prof_Moderate,prof_No Rate,prof_None,prof_Severe
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,1,1,0,0,0,0
1,The Guilty,"Crime, Drama, Thriller",R,3,3,0,0,0,0,1
2,The Many Saints of Newark,"Crime, Drama",R,1,4,0,0,0,0,1
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,1,0,1,0,0,0
4,Dune,"Action, Adventure, Drama",PG-13,3,1,0,0,0,1,0


In [ ]:
cat_imdb_df_1[["prof_Mild", "prof_Moderate"]].dtypes

prof_Mild        uint8
prof_Moderate    uint8
dtype: object

In [ ]:
# ints to one-hot

cat_imdb_df_1 = pd.get_dummies(cat_imdb_df_1, columns=["Violence"], prefix=["viol"])
cat_imdb_df_1.head()

,Name,Genre,Certificate,Nudity,prof_Mild,prof_Moderate,prof_No Rate,prof_None,prof_Severe,viol_0,viol_1,viol_2,viol_3,viol_4
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,1,0,0,0,0,0,1,0,0,0
1,The Guilty,"Crime, Drama, Thriller",R,3,0,0,0,0,1,0,0,0,1,0
2,The Many Saints of Newark,"Crime, Drama",R,1,0,0,0,0,1,0,0,0,0,1
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,0,1,0,0,0,0,1,0,0,0
4,Dune,"Action, Adventure, Drama",PG-13,3,0,0,0,1,0,0,1,0,0,0


In [ ]:
# Custom Binary encoding

import numpy as np

cat_imdb_df_1["Genre"] = np.where(cat_imdb_df_1["Genre"].str.contains("Drama"), 1, 0)
cat_imdb_df_1.head()

,Name,Genre,Certificate,Nudity,prof_Mild,prof_Moderate,prof_No Rate,prof_None,prof_Severe,viol_0,viol_1,viol_2,viol_3,viol_4
0,No Time to Die,0,PG-13,0,1,0,0,0,0,0,1,0,0,0
1,The Guilty,1,R,3,0,0,0,0,1,0,0,0,1,0
2,The Many Saints of Newark,1,R,1,0,0,0,0,1,0,0,0,0,1
3,Venom: Let There Be Carnage,0,PG-13,3,0,1,0,0,0,0,1,0,0,0
4,Dune,1,PG-13,3,0,0,0,1,0,0,1,0,0,0


In [ ]:
# It should be noted - we have default numpy precision here - int64
cat_imdb_df_1["Genre"].dtype

dtype('int64')

# Sklearn facilities for encoding of categorical features

In [ ]:
cat_imdb_df_2 = cat_imdb_df.copy()

cat_imdb_df_2.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,Mild,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,None,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,Moderate,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,None,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,None,Moderate,None


In [ ]:
# Categorical features to ints

from sklearn.preprocessing import LabelEncoder

# This transformer should be used to encode target values, i.e. y, and not the inputs.
label_encoder = LabelEncoder()
cat_imdb_df_2["Nudity"] = label_encoder.fit_transform(cat_imdb_df_2["Nudity"])
cat_imdb_df_2.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,3,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,1,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,3,Moderate,None


In [ ]:
cat_imdb_df_2["Nudity"].dtype

dtype('int64')

In [ ]:
# Categorical features to floats

from sklearn.preprocessing import OrdinalEncoder

cat_imdb_df_2_1 = cat_imdb_df.copy()

# This transformer should be used to encode target values, i.e. y, and not the inputs.
ordinal_encoder = OrdinalEncoder()

cat_nudity_array = cat_imdb_df_2_1["Nudity"].to_numpy()
cat_nudity_array = cat_nudity_array.reshape(-1, 1)
cat_nudity_array

array([['Mild'],
       ['None'],
       ['Moderate'],
       ...,
       ['Severe'],
       ['None'],
       ['Mild']], dtype=object)

In [ ]:
cat_imdb_df_2_1["Nudity"] = ordinal_encoder.fit_transform(cat_nudity_array)
cat_imdb_df_2_1.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0.0,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,3.0,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,1.0,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3.0,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,3.0,Moderate,None


In [ ]:
cat_imdb_df_2_1["Nudity"].dtype

dtype('float64')

In [ ]:
# Categorical feature to one-hot

from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
lb_encoded = label_binarizer.fit_transform(cat_imdb_df_2["Profanity"])
new_df = pd.DataFrame(lb_encoded, columns=label_binarizer.classes_)
new_df.head()

,Mild,Moderate,No Rate,None,Severe
0,1,0,0,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,1,0,0,0
4,0,0,0,1,0


In [ ]:
# Concatenate two DataFrames

cat_imdb_df_3 = pd.concat([cat_imdb_df_2, new_df], axis=1)
cat_imdb_df_3.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity,Mild,Moderate,No Rate,None,Severe
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,Moderate,Mild,1,0,0,0,0
1,The Guilty,"Crime, Drama, Thriller",R,3,None,Severe,0,0,0,0,1
2,The Many Saints of Newark,"Crime, Drama",R,1,Severe,Severe,0,0,0,0,1
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,Moderate,Moderate,0,1,0,0,0
4,Dune,"Action, Adventure, Drama",PG-13,3,Moderate,None,0,0,0,1,0


In [ ]:
cat_imdb_df_3.drop(columns=['Profanity'], inplace=True)
cat_imdb_df_3.head()

,Name,Genre,Certificate,Nudity,Violence,Mild,Moderate,No Rate,None,Severe
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,Moderate,1,0,0,0,0
1,The Guilty,"Crime, Drama, Thriller",R,3,None,0,0,0,0,1
2,The Many Saints of Newark,"Crime, Drama",R,1,Severe,0,0,0,0,1
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,Moderate,0,1,0,0,0
4,Dune,"Action, Adventure, Drama",PG-13,3,Moderate,0,0,0,1,0


In [ ]:
cat_imdb_df_3["Mild"].dtype

dtype('int64')

In [ ]:
# Encoding to one-hot using OneHotEncoder

cat_profanity_array = cat_imdb_df_2["Profanity"].to_numpy()
cat_profanity_array = cat_profanity_array.reshape(-1, 1)
cat_profanity_array

array([['Mild'],
       ['Severe'],
       ['Severe'],
       ...,
       ['Severe'],
       ['Mild'],
       ['Severe']], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False)

categories_encoded = one_hot_encoder.fit_transform(cat_profanity_array)
one_hot_encoder.categories_

[array(['Mild', 'Moderate', 'No Rate', 'None', 'Severe'], dtype=object)]

In [ ]:
new_df2 = pd.DataFrame(categories_encoded, columns=one_hot_encoder.categories_[0])
new_df2.head()

,Mild,Moderate,No Rate,None,Severe
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0


In [ ]:
new_df2["Mild"].dtype

dtype('float64')

One of the big benefits of using OneHotEncoder with Scikit Learn models is that you can include it, along with the model itself, as a step in a Scikit Learn Pipeline, essentially bundling One Hot Encoding (and potentially other preprocessing) logic and inference logic as a single deployable artifact.

In [ ]:
cleanup_nums = {"Profanity" : {
    "Mild": 0,
    "Moderate": 3,
    "No Rate": 2,
    "None": 4,
    "Severe": 1,
    },
    }
cat_imdb_df_4 = cat_imdb_df_2.copy()
cat_imdb_df_4.replace(cleanup_nums, inplace=True)
labels = cat_imdb_df_4["Profanity"]
labels

0       0
1       1
2       1
3       3
4       4
       ..
6173    1
6174    4
6175    1
6176    0
6177    1
Name: Profanity, Length: 6178, dtype: int64

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

one_hot_encoder = OneHotEncoder(sparse=False)

linear_regression = LinearRegression()
pipeline = Pipeline(steps=[('one_hot_encoder',one_hot_encoder), ('linear_regression', linear_regression)])
pipeline.fit(cat_profanity_array, labels)

print('Severe Prediction:', pipeline.predict([['Severe']])[0])
print('Moderate Prediction:', pipeline.predict([['Moderate']])[0])

Severe Prediction: 1.0000000000000004
Moderate Prediction: 3.0


Back to our TensorFlow scenario: if you want to use OneHotEncoder to preprocess input features for a TensorFlow model, you may have some additional complexity to deal with, because you have to:
1. Duplicate the One Hot Encoding logic anywhere that the model is used for inference.
2. Or, deploy both the fit OneHotEncoder and the trained TensorFlow model as separate artifacts, and then ensure that they are used properly and kept in-sync by all applications that use the model.

# Sklearn for bag-of-words, n-grams, tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# class sklearn.feature_extraction.text.CountVectorizer(*, input='content', encoding='utf-8',
# decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None,
# tokenizer=None, stop_words=None, token_pattern='(?u)\b\w\w+\b', ngram_range=(1, 1),
# analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None,
# binary=False, dtype=<class 'numpy.int64'>)[source]

# input='content', the input is expected to be a sequence of items that can be of type string or byte.

# strip_accents: {‘ascii’, ‘unicode’}, default=None
# Remove accents and perform other character normalization during the preprocessing step.
# ‘ascii’ is a fast method that only works on characters that have an direct ASCII mapping.
# ‘unicode’ is a slightly slower method that works on any characters.
# None (default) does nothing.

# lowercase: bool, default=True
# Convert all characters to lowercase before tokenizing.

# preprocessor: callable, default=None
# Override the preprocessing (strip_accents and lowercase) stage while preserving the tokenizing
# and n-grams generation steps. Only applies if analyzer is not callable.

# tokenizer: callable, default=None
# Override the string tokenization step while preserving the preprocessing
# and n-grams generation steps. Only applies if analyzer == 'word'.

# stop_words: {‘english’}, list, default=None
# If ‘english’, a built-in stop word list for English is used.
# There are several known issues with ‘english’ and you should consider an alternative.
# If a list, that list is assumed to contain stop words,
# all of which will be removed from the resulting tokens.
# Only applies if analyzer == 'word'.
# If None, no stop words will be used. max_df can be set to a value in the range [0.7, 1.0)
# to automatically detect and filter stop words based on intra corpus document frequency of terms.

# token_pattern: str, default=r”(?u)\b\w\w+\b”
# Regular expression denoting what constitutes a “token”, only used if analyzer == 'word'.
# The default regexp select tokens of 2 or more alphanumeric characters
# (punctuation is completely ignored and always treated as a token separator).
# If there is a capturing group in token_pattern then the captured group content,
# not the entire match, becomes the token. At most one capturing group is permitted.

# ngram_range: tuple (min_n, max_n), default=(1, 1)
# The lower and upper boundary of the range of n-values for different word n-grams
# or char n-grams to be extracted.
# All values of n such that min_n <= n <= max_n will be used.
# For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams,
# and (2, 2) means only bigrams. Only applies if analyzer is not callable.

# analyzer: {‘word’, ‘char’, ‘char_wb’} or callable, default=’word’
# Whether the feature should be made of word n-gram or character n-grams.
# Option ‘char_wb’ creates character n-grams only from text inside word boundaries;
# n-grams at the edges of words are padded with space.
# If a callable is passed it is used to extract the sequence of features out of the raw, unprocessed input.

# Если 'word', то в словарь добавляются сочетания слов, количество слов задаётся в ngram_range.
# Если 'char_wb', то в словарь добавляются сочетания символов, затрагивающие одно слово и пробелы вокруг слова,
# количество символов в сочетаниях задаётся в ngram_range.
# Если 'char', то то в словарь добавляются сочетания символов, в том числе затрагивающие несколько слов,
# пробелы, знаки пунктуации.
# Количество символов в сочетаниях задаётся в ngram_range.

# max_df: float in range [0.0, 1.0] or int, default=1.0
# When building the vocabulary ignore terms that have a document frequency strictly higher than
# the given threshold (corpus-specific stop words).
# If float, the parameter represents a proportion of documents, integer absolute counts.
# This parameter is ignored if vocabulary is not None.

# min_df: float in range [0.0, 1.0] or int, default=1
# When building the vocabulary ignore terms that have a document frequency strictly lower than
# the given threshold.
# This value is also called cut-off in the literature.
# If float, the parameter represents a proportion of documents, integer absolute counts.
# This parameter is ignored if vocabulary is not None.

# max_features: int, default=None
# If not None, build a vocabulary that only consider the top max_features
# ordered by term frequency across the corpus.
# This parameter is ignored if vocabulary is not None.

# vocabulary: Mapping or iterable, default=None
# Either a Mapping (e.g., a dict) where keys are terms and values are indices in the feature matrix,
# or an iterable over terms.
# If not given, a vocabulary is determined from the input documents.
# Indices in the mapping should not be repeated and should not have any gap between
# 0 and the largest index.

# binary: bool, default=False
# If True, all non zero counts are set to 1. This is useful for discrete probabilistic models
# that model binary events rather than integer counts.

# dtype: type, default=np.int64
# Type of the matrix returned by fit_transform() or transform().

# При использовании конструктора по умолчанию объект  CountVectorizer удаляет пунктуацию,
# односимвольные токены,  игнорирует регистр.


count_vectorizer = CountVectorizer()

# Create the Bag-of-Words
bag_of_words = count_vectorizer.fit_transform(cat_imdb_df_2["Name"])
bag_of_words

<6178x4919 sparse matrix of type '<class 'numpy.int64'>'
	with 15262 stored elements in Compressed Sparse Row format>

In [ ]:
# Как видим, результат получается в виде sparse matrix

# Для преобразования в обычный ndarray
bag_of_words.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# Как видим, каждая из 6178 строк представлена в виде вектора длиной 4919
# Здесь 4919 - размер словаря (уникальных слов), который определён на базе исходных строк
bag_of_words.toarray().shape

(6178, 4919)

In [ ]:
# Можно ограничить размер словаря, например, 1000 наиболее частотных слов

count_vectorizer = CountVectorizer(max_features=1000)

bag_of_words = count_vectorizer.fit_transform(cat_imdb_df_2["Name"])
bag_of_words.toarray().shape

(6178, 1000)

In [ ]:
# Можно создавать словарь из n-gram

count_vectorizer = CountVectorizer(ngram_range=(5,5), analyzer='char_wb')

bag_of_words = count_vectorizer.fit_transform(cat_imdb_df_2["Name"])
bag_of_words.toarray().shape

(6178, 16716)

In [ ]:
# Просмотр элементов словаря
count_vectorizer.get_feature_names()

[' #sar',
 ' & ',
 " '66 ",
 " '70s",
 " '80s",
 " 'all",
 " 'bur",
 " 'em ",
 " 'r ",
 ' (500',
 ' (fas',
 ' (fin',
 ' (fir',
 ' (ful',
 ' (ii)',
 ' (ish',
 ' (the',
 ' + ',
 ' - ',
 ' 1 ',
 ' 1/2 ',
 ' 10 ',
 ' 100 ',
 ' 1000',
 ' 101 ',
 ' 12 ',
 ' 12. ',
 ' 120 ',
 ' 127 ',
 ' 13 ',
 ' 13th',
 ' 1408',
 ' 16 ',
 ' 1666',
 ' 17 ',
 ' 1883',
 ' 19 ',
 ' 1917',
 ' 1922',
 ' 1978',
 ' 1984',
 ' 1992',
 ' 1994',
 ' 1: ',
 ' 2 ',
 ' 2, ',
 ' 20 ',
 ' 2000',
 ' 2001',
 ' 2010',
 ' 2012',
 ' 2049',
 ' 2067',
 ' 21 ',
 ' 22 ',
 ' 24 ',
 ' 25th',
 ' 26/1',
 ' 27 ',
 ' 28 ',
 ' 2: ',
 ' 2u ',
 ' 3 ',
 ' 3% ',
 ' 3-d ',
 ' 30 ',
 ' 300 ',
 ' 300:',
 ' 31 ',
 ' 355 ',
 ' 365 ',
 ' 37°2',
 ' 39 ',
 ' 3: ',
 ' 3:10',
 ' 3d ',
 ' 3dd ',
 ' 3rd ',
 ' 4 ',
 ' 40 ',
 ' 404 ',
 ' 414 ',
 ' 42 ',
 ' 43 ',
 ' 44 ',
 ' 4400',
 ' 47 ',
 ' 4: ',
 ' 5 ',
 ' 50 ',
 ' 5: ',
 ' 5th ',
 ' 6 ',
 ' 68 ',
 ' 7 ',
 ' 7th ',
 ' 8 ',
 ' 84 ',
 ' 86 ',
 ' 8mm ',
 ' 9 ',
 ' 9-1-',
 ' 9021',
 ' 911!',
 ' 93 ',
 ' 99 ',


In [ ]:
# Маппинг элементов словаря к их индексам в словаре (к их индексам в каждом выходном векторе фичи)
count_vectorizer.vocabulary_

{' no ': 2211,
 ' time': 3129,
 'time ': 15227,
 ' to ': 3135,
 ' die ': 858,
 ' the ': 3100,
 ' guil': 1362,
 'guilt': 8035,
 'uilty': 15679,
 'ilty ': 8924,
 ' many': 1959,
 'many ': 10512,
 ' sain': 2699,
 'saint': 14118,
 'aints': 3995,
 'ints ': 9137,
 ' of ': 2269,
 ' newa': 2196,
 'newar': 11198,
 'ewark': 7367,
 'wark ': 16256,
 ' veno': 3290,
 'venom': 16103,
 'enom:': 6914,
 'nom: ': 11442,
 ' let ': 1818,
 ' ther': 3106,
 'there': 15142,
 'here ': 8273,
 ' be ': 351,
 ' carn': 584,
 'carna': 5387,
 'arnag': 4569,
 'rnage': 13737,
 'nage ': 10966,
 ' dune': 948,
 'dune ': 6306,
 ' ted ': 3079,
 ' lass': 1789,
 'lasso': 9864,
 'asso ': 4681,
 ' free': 1205,
 'free ': 7640,
 ' guy ': 1371,
 ' hous': 1496,
 'house': 8488,
 'ouse ': 12489,
 ' drag': 921,
 'drago': 6253,
 'ragon': 13111,
 'agon ': 3940,
 ' what': 3396,
 'what ': 16319,
 ' if..': 1532,
 'if...': 8775,
 'f...?': 7428,
 '...? ': 3622,
 ' sex ': 2787,
 ' educ': 975,
 'educa': 6545,
 'ducat': 6290,
 'ucati': 15590,
 'c

In [ ]:
# Сколько максимально раз одна n-gram встречается в одной и той же строке
np.max(bag_of_words.toarray())

4

In [ ]:
# Каждая строка (значение фичи) представляется как вектор длиной = (размер словаря)
# Каждый элемент вектора - число:
# 0 если в данной строке нет такого слова из словаря.
# 1 если в данной строке 1 раз встречается такое слово из словаря.
# 2 если в данной строке 2 раза встречается такое слово из словаря, и т.д.
# Если задать аргумент binary=True, то количество раз не в счёт, каждый элемент или 0 или 1.

count_vectorizer = CountVectorizer(ngram_range=(5,5), analyzer='char_wb', binary=True)

bag_of_words = count_vectorizer.fit_transform(cat_imdb_df_2["Name"])
bag_of_words.toarray().shape

(6178, 16716)

In [ ]:
np.max(bag_of_words.toarray())

1

In [ ]:
count_vectorizer = CountVectorizer()

# Create the Bag-of-Words
bag_of_words = count_vectorizer.fit_transform(cat_imdb_df_2["Genre"])
bag_of_words.toarray().shape

(6178, 31)

In [ ]:
count_vectorizer.get_feature_names()

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'fi',
 'film',
 'game',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'noir',
 'reality',
 'romance',
 'sci',
 'short',
 'show',
 'sport',
 'talk',
 'thriller',
 'tv',
 'war',
 'western']

In [ ]:
count_vectorizer.vocabulary_

{'action': 0,
 'adventure': 1,
 'animation': 2,
 'biography': 3,
 'comedy': 4,
 'crime': 5,
 'documentary': 6,
 'drama': 7,
 'family': 8,
 'fantasy': 9,
 'fi': 10,
 'film': 11,
 'game': 12,
 'history': 13,
 'horror': 14,
 'music': 15,
 'musical': 16,
 'mystery': 17,
 'news': 18,
 'noir': 19,
 'reality': 20,
 'romance': 21,
 'sci': 22,
 'short': 23,
 'show': 24,
 'sport': 25,
 'talk': 26,
 'thriller': 27,
 'tv': 28,
 'war': 29,
 'western': 30}

In [ ]:
first_record = bag_of_words.toarray()[0]
first_record
# 'action': 0, 'adventure': 1, 'thriller': 27

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
second_record = bag_of_words.toarray()[1]
second_record
# 'crime': 5, 'drama': 7, 'thriller': 27

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,fi,film,game,history,horror,music,musical,mystery,news,noir,reality,romance,sci,short,show,sport,talk,thriller,tv,war,western
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6173,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6174,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6175,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6176,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF
tfidf_vectorizer = TfidfVectorizer()

tfidf_result = tfidf_vectorizer.fit_transform(cat_imdb_df_2["Genre"])
tfidf_result.toarray().shape

(6178, 31)

In [ ]:
tfidf_vectorizer.get_feature_names()

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'fi',
 'film',
 'game',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'noir',
 'reality',
 'romance',
 'sci',
 'short',
 'show',
 'sport',
 'talk',
 'thriller',
 'tv',
 'war',
 'western']

In [ ]:
tfidf_vectorizer.vocabulary_

{'action': 0,
 'adventure': 1,
 'animation': 2,
 'biography': 3,
 'comedy': 4,
 'crime': 5,
 'documentary': 6,
 'drama': 7,
 'family': 8,
 'fantasy': 9,
 'fi': 10,
 'film': 11,
 'game': 12,
 'history': 13,
 'horror': 14,
 'music': 15,
 'musical': 16,
 'mystery': 17,
 'news': 18,
 'noir': 19,
 'reality': 20,
 'romance': 21,
 'sci': 22,
 'short': 23,
 'show': 24,
 'sport': 25,
 'talk': 26,
 'thriller': 27,
 'tv': 28,
 'war': 29,
 'western': 30}

In [ ]:
# Основное отличие TfidfVectorizer от CountVectorizer в том, что он возвращает
# scores (во float), а не counts (в int)

first_record = tfidf_result.toarray()[0]
first_record
# 'action': 0, 'adventure': 1, 'thriller': 27

array([0.51936916, 0.57543947, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.63176348, 0.        , 0.        ,
       0.        ])

In [ ]:
second_record = tfidf_result.toarray()[1]
second_record
# 'crime': 5, 'drama': 7, 'thriller': 27

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.62869917, 0.        , 0.39101577, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.67219345, 0.        , 0.        ,
       0.        ])

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(tfidf_result.toarray(), columns = feature_names)

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,fi,film,game,history,horror,music,musical,mystery,news,noir,reality,romance,sci,short,show,sport,talk,thriller,tv,war,western
0,0.519369,0.575439,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.631763,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.000000,0.628699,0.0,0.391016,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.672193,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.000000,0.849163,0.0,0.528132,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.392580,0.434963,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.573013,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.573013,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.605422,0.670782,0.0,0.0,0.000000,0.000000,0.0,0.428387,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6173,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6174,0.000000,0.000000,0.0,0.0,0.000000,0.233971,0.0,0.145517,0.0,0.0,0.000000,0.679736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.679736,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6175,0.000000,0.000000,0.0,0.0,0.000000,0.628699,0.0,0.391016,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.672193,0.0,0.0,0.0
6176,0.000000,0.000000,0.0,0.0,0.519895,0.000000,0.0,0.399047,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.755295,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


## Encoding categorical features in TensorFlow

In [ ]:
cat_imdb_df_5 = cat_imdb_df_2.copy()
cat_imdb_df_5.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,0,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,3,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,1,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,3,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,3,Moderate,None


In [ ]:
cat_imdb_df_5["Violence"].unique()

array(['Moderate', 'None', 'Severe', 'No Rate', 'Mild'], dtype=object)

In [ ]:
number_unique_categories = len(cat_imdb_df_5["Violence"].unique())

In [ ]:
indices = tf.range(number_unique_categories, dtype=tf.int64)
indices

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [ ]:
vocab = cat_imdb_df_5["Violence"].unique()
keys_tensor = tf.constant(vocab)
keys_tensor

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Moderate', b'None', b'Severe', b'No Rate', b'Mild'], dtype=object)>

In [ ]:
table_init = tf.lookup.KeyValueTensorInitializer(keys_tensor, indices)

In [ ]:
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [ ]:
# Encoding to ints

# There are indices for unknown words due to oov_buckets
category_values = tf.constant(['Moderate', 'Sex', 'Severe'])
cat_indices = table.lookup(category_values)
cat_indices

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 6, 2])>

In [ ]:
category_values = tf.constant(cat_imdb_df_5["Violence"])
cat_indices = table.lookup(category_values)
cat_indices

<tf.Tensor: shape=(6178,), dtype=int64, numpy=array([0, 1, 2, ..., 0, 1, 2])>

In [ ]:
# Encoding to one-hot

# tf.one_hot does not accept the categories themselves, but instead accepts a list of indices
cat_one_hot = tf.one_hot(cat_indices, depth=number_unique_categories+num_oov_buckets )
cat_one_hot

<tf.Tensor: shape=(6178, 7), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

In [ ]:
# Verify for 'Moderate' - 1 0 0 0 0 0 0
cat_one_hot[4]

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
# tf.keras.preprocessing.text.one_hot

# It's hard to name this one-hot, but nevertheless there is the such function

tf.keras.preprocessing.text.one_hot("Some string", 10)

[8, 6]

In [ ]:
tf.keras.preprocessing.text.one_hot("Some string", 500)

[53, 141]

In [ ]:
# It's more likely than not to use it as preprocessing for one-hot

size_vocabulary = 10

list_ints_1 = tf.keras.preprocessing.text.one_hot("Some string", size_vocabulary)
list_ints_2 = tf.keras.preprocessing.text.one_hot("Another message", size_vocabulary)

cat_one_hot = tf.one_hot(list_ints_1, depth=size_vocabulary)
cat_one_hot

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)>

In [ ]:
cat_one_hot = tf.one_hot(list_ints_2, depth=size_vocabulary)
cat_one_hot

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)>

### Encoding categorical features using TextVectorization

In [ ]:
cat_imdb_df_6 = cat_imdb_df.copy()
cat_imdb_df_6.head()

,Name,Genre,Certificate,Nudity,Violence,Profanity
0,No Time to Die,"Action, Adventure, Thriller",PG-13,Mild,Moderate,Mild
1,The Guilty,"Crime, Drama, Thriller",R,None,None,Severe
2,The Many Saints of Newark,"Crime, Drama",R,Moderate,Severe,Severe
3,Venom: Let There Be Carnage,"Action, Adventure, Sci-Fi",PG-13,None,Moderate,Moderate
4,Dune,"Action, Adventure, Drama",PG-13,None,Moderate,None


In [ ]:
# tf.keras.layers.TextVectorization(
#     max_tokens=None, standardize='lower_and_strip_punctuation',
#     split='whitespace', ngrams=None, output_mode='int',
#     output_sequence_length=None, pad_to_max_tokens=False, vocabulary=None, **kwargs
# )

# It transforms a batch of strings (one example = one string) into either a list
# of token indices (one example = 1D tensor of integer token indices) or
# a dense representation (one example = 1D tensor of float values
# representing data about the example's tokens).

# If desired, the user can call this layer's adapt() method on a dataset.
# When this layer is adapted, it will analyze the dataset,
# determine the frequency of individual string values,
# and create a 'vocabulary' from them.
# This vocabulary can have unlimited size or be capped,
# depending on the configuration options for this layer;
# if there are more unique values in the input than the maximum vocabulary size,
# the most frequent terms will be used to create the vocabulary.

# The processing of each example contains the following steps:

  # Standardize each example (usually lowercasing + punctuation stripping);
  # Split each example into substrings (usually words);
  # Recombine substrings into tokens (usually ngrams);
  # Index tokens (associate a unique int value with each token);
  # Transform each example using this index, either into a vector of ints
  # or a dense float vector.

# max_tokens - The maximum size of the vocabulary for this layer.
# If None, there is no cap on the size of the vocabulary.

# standardize - Optional specification for standardization to apply to the input text.
# Values can be None (no standardization),
# "lower_and_strip_punctuation" (lowercase and remove punctuation) or a Callable.
# Default is "lower_and_strip_punctuation".

# split - Optional specification for splitting the input text.
# Values can be None (no splitting), "whitespace" (split on ASCII whitespace),
# or a Callable.
# The default is "whitespace".

# ngrams - Optional specification for ngrams to create from the possibly-split input text.
# Values can be None, an integer or tuple of integers;
# passing an integer will create ngrams up to that integer,
# and passing a tuple of integers will create ngrams for the specified values in the tuple.
# Passing None means that no ngrams will be created.

# output_mode:

    # "int": Outputs integer indices, one integer index per split string token.
    # When output_mode == "int", 0 is reserved for masked locations;
    # this reduces the vocab size to max_tokens - 2 instead of max_tokens - 1.

    # "multi_hot": Outputs a single int array per batch,
    # of either vocab_size or max_tokens size,
    # containing Ones in all elements where the token mapped to that index exists
    # at least once in the batch item.

    # "count": Like "multi_hot", but the int array contains a count of the number
    # of times the token at that index appeared in the batch item.

    # "tf_idf": Like "multi_hot", but the TF-IDF algorithm is applied to find
    # the value in each token slot.
    
    # For "int" output, any shape of input and output is supported.
    # For all other output modes, currently only rank 1 inputs
    # (and rank 2 outputs after splitting) are supported.

# output_sequence_length - Only valid in INT mode. If set, the output will have
# its time dimension padded or truncated to exactly output_sequence_length values,
# resulting in a tensor of shape (batch_size, output_sequence_length) regardless
# of how many tokens resulted from the splitting step. Defaults to None.

# pad_to_max_tokens - Only valid in "multi_hot", "count", and "tf_idf" modes.
# If True, the output will have its feature axis padded to max_tokens even if
# the number of unique tokens in the vocabulary is less than max_tokens,
# resulting in a tensor of shape (batch_size, max_tokens) regardless of vocabulary size.
# Defaults to False.

# vocabulary - Optional. Either an array of strings or a string path to a text file.


text_vectorizer = tf.keras.layers.TextVectorization()
text_vectorizer.adapt(cat_imdb_df_6['Nudity'])
text_vectorizer.call(cat_imdb_df_6['Violence'])

<tf.Tensor: shape=(6178, 2), dtype=int64, numpy=
array([[4, 0],
       [3, 0],
       [7, 0],
       ...,
       [4, 0],
       [3, 0],
       [7, 0]])>

In [ ]:
# Как видим, по умолчанию строки разделяются на слова (дефолтное разделение - пробел)
# Каждое уникальное слово (в пределах батча) в обучающей выборке
# получает свой уникальный индекс (целое число)
# Один индекс (1 oov-bucket) выделяется для неизвестных слов

text_vectorizer.get_vocabulary()

['', '[UNK]', 'mild', 'none', 'moderate', 'rate', 'no', 'severe']

In [ ]:
# Каждый вектор содержит 0 в позициях, для которых не найдено слов в словаре
# или число (числа), соответствующие найденным индексам в словаре.
# Размер выходных векторов определяется размером самой длинной строки среди строк
# на входе метода call().
# В данном случае максимальная строка - из двух слов.

# Вот та самая строка из двух слов в выборке cat_imdb_df_6['Violence'] 
text_vectorizer.call(['No Rate'])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[6, 5]])>

In [ ]:
# Одно слово найдено
text_vectorizer.call(['Rate'])

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[5]])>

In [ ]:
# Два слова найдены
text_vectorizer.call(['Rate Severe'])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[5, 7]])>

In [ ]:
# Обращаем внимание, что если на входе call() только строки длиной 1,
# то и выходные вектора имеют длину 1
text_vectorizer.call([['Rate'], ['No']])

<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[5],
       [6]])>

In [ ]:
# А если хотя бы одна из входных строк длиннее других,
# то ширина выходной матрицы равна максимальной длине строки
text_vectorizer.call([['Rate'], ['No Rate']])

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[5, 0],
       [6, 5]])>

In [ ]:
# Незнакомое слово найдено
text_vectorizer.call(['Rate Good'])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[5, 1]])>

In [ ]:
# Можно задать явно размер выходного вектора (только для output_mode='int')

text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=1)
text_vectorizer.adapt(cat_imdb_df_6['Nudity'])
text_vectorizer.call(cat_imdb_df_6['Violence'])

<tf.Tensor: shape=(6178, 1), dtype=int64, numpy=
array([[4],
       [3],
       [7],
       ...,
       [4],
       [3],
       [7]])>

In [ ]:
# И тогда выходной вектор или дополняется нулями справа (если строка короче)
# или усекается (остаются только крайние левые столбцы), т.е. лишние (справа) слова отбрасываются
text_vectorizer.call(['No Rate'])

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[6]])>

In [ ]:
text_vectorizer.call(['Rate Severe'])

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[5]])>

In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=10)
text_vectorizer.adapt(cat_imdb_df_6['Nudity'])
text_vectorizer.call(cat_imdb_df_6['Violence'])

<tf.Tensor: shape=(6178, 10), dtype=int64, numpy=
array([[4, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0],
       ...,
       [4, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0]])>

In [ ]:
# В режиме multi_hot получаем представление строк в виде bag-of-words,
# но в режиме binary=True.
# Как видим, здесь длина вектора зависит от длины словаря

text_vectorizer = tf.keras.layers.TextVectorization(output_mode='multi_hot')
text_vectorizer.adapt(cat_imdb_df_6['Nudity'])
text_vectorizer.call(cat_imdb_df_6['Violence'])

<tf.Tensor: shape=(6178, 7), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Здесь в строке 2 слова из словаря и одно незнакомое слово (размер словаря - 7)
# Одно из слов встречается 2 раза
text_vectorizer.call([['Mild Moderate Sex Mild']])

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=array([[1., 1., 0., 1., 0., 0., 0.]], dtype=float32)>

In [ ]:
# В режиме count получаем представление строк в виде bag-of-words
# Как видим, здесь длина вектора зависит от длины словаря

text_vectorizer = tf.keras.layers.TextVectorization(output_mode='count')
text_vectorizer.adapt(cat_imdb_df_6['Nudity'])
text_vectorizer.call(cat_imdb_df_6['Violence'])

<tf.Tensor: shape=(6178, 7), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Здесь в строке 2 слова из словаря и одно незнакомое слово (размер словаря - 7)
# И одно слово встречается 2 раза (count=2)
text_vectorizer.call([['Mild Moderate Sex Mild']])

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=array([[1., 2., 0., 1., 0., 0., 0.]], dtype=float32)>

In [ ]:
# На выходе слоя TextVectorization получаем обычный тензор, не sparse tensor
# В дефолтном режиме 'int' длина вектора равна максимальной длине строки
text_vectorizer = tf.keras.layers.TextVectorization()
text_vectorizer.adapt(cat_imdb_df_6['Name'])
result = text_vectorizer.call(cat_imdb_df_6['Name'])
result

<tf.Tensor: shape=(6178, 18), dtype=int64, numpy=
array([[  10,   39,    8, ...,    0,    0,    0],
       [   2,  708,    0, ...,    0,    0,    0],
       [   2, 3603, 1508, ...,    0,    0,    0],
       ...,
       [ 158,    2,  268, ...,    0,    0,    0],
       [ 147, 2356,    0, ...,    0,    0,    0],
       [  70,  368,    0, ...,    0,    0,    0]])>

In [ ]:
# В режиме 'multi_hot' длина вектора равна длине словаря
text_vectorizer = tf.keras.layers.TextVectorization(output_mode='multi_hot')
text_vectorizer.adapt(cat_imdb_df_6['Name'])
result = text_vectorizer.call(cat_imdb_df_6['Name'])
result

<tf.Tensor: shape=(6178, 5012), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>